# 在 Google Colab 中进行 EchoHeart 的 Qwen 微调 (可配置版 v2)

这个 Notebook 会自动配置环境、启动微调 (QLoRA)、运行测试、合并 LoRA 适配器并导出 GGUF 模型。

**配置**：您可以通过修改下面的第一个代码单元格中的变量来指定要使用的基础模型和数据集。

**步骤：**
1. 配置变量、克隆/更新 GitHub 仓库并定义路径。
2. 安装必要的依赖项。
3. 运行训练脚本 (QLoRA)。
4. (可选) 运行测试脚本与微调后的模型(适配器)交互。
5. 合并 LoRA 适配器到基础模型。
6. (可选) 将合并后的模型转换为 GGUF 格式。

In [ ]:
# 1. 配置变量、克隆/更新仓库并定义路径
import os

# --- 配置变量 (在此处修改！) ---
# 指定要微调的基础模型 (Hugging Face 名称或路径)
base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
# 指定数据集文件路径 (相对于仓库根目录)
dataset_file = "data/converted_dataset.json" # 确保这个文件存在于您的仓库中
# ------------------------------

# --- 自动生成路径 (基于上面的配置) ---
model_name_slug = base_model_name.split('/')[-1] # 从模型名称生成安全的文件名部分
adapter_output_dir = f"output/{model_name_slug}-qlora-ft"
merged_model_output_dir = f"output/{model_name_slug}-merged-ft"
gguf_output_filename = f"gguf-{model_name_slug}-f16.gguf" # GGUF文件名
gguf_output_dir = merged_model_output_dir # GGUF文件保存在合并模型的目录中
# -------------------------------------

# -- 固定路径和计算绝对路径 --
repo_path = '/content/echoheart_demo'
adapter_path = os.path.join(repo_path, adapter_output_dir)
merged_model_path = os.path.join(repo_path, merged_model_output_dir)
dataset_abs_path = os.path.join(repo_path, dataset_file)
gguf_output_abs_path = os.path.join(repo_path, gguf_output_dir, gguf_output_filename) 
# --------------

print("--- 配置信息 ---")
print(f"基础模型: {base_model_name}")
print(f"数据集文件 (相对路径): {dataset_file}")
print(f"数据集文件 (绝对路径): {dataset_abs_path}")
print(f"适配器输出目录: {adapter_path}")
print(f"合并模型输出目录: {merged_model_path}")
print(f"GGUF 输出文件: {gguf_output_abs_path}")
print("------------------")

# 克隆或更新仓库
if not os.path.exists(repo_path):
  print(f"\nCloning repository into {repo_path}...")
  !git clone https://github.com/shuakami/echoheart_demo.git {repo_path}
else:
  print(f"\nRepository already exists at {repo_path}.")

%cd {repo_path}

print("Pulling latest changes...")
!git pull origin master

# 确保数据集文件存在
if not os.path.exists(dataset_abs_path):
  print(f"\n[bold red]错误：指定的数据集文件不存在: {dataset_abs_path}[/bold red]")
  # 可以在这里引发错误停止执行: raise FileNotFoundError(f"Dataset file not found: {dataset_abs_path}")
else:
  print(f"\n数据集文件确认存在: {dataset_abs_path}")

!pwd

In [ ]:
# 2. 安装依赖
print("Installing dependencies from requirements.txt...")
!pip install -q -r requirements.txt

print("Applying dependency fixes...")
!pip install -q fsspec==2024.12.0 # 使用 -q 安静模式

In [ ]:
# 3. 运行训练脚本 (QLoRA)
print("Starting QLoRA training...")
# 通过命令行参数传递模型名称和数据集路径 (使用变量)
!python train.py --base_model_name "{base_model_name}" --dataset_file "{dataset_file}"

## 训练完成！

QLoRA 适配器应保存在 Colab 文件系统的 `{adapter_path}` 目录中。"

In [ ]:
# 4. (可选) 运行测试脚本 (测试适配器效果)
print("Starting non-interactive testing session with adapter...")
# 使用变量调用测试脚本
!python test_model.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}"

## 5. 合并 LoRA 适配器

以下步骤将加载基础模型和训练好的 LoRA 适配器，并将它们合并成一个完整的模型。
合并后的模型将保存在 `{merged_model_path}`。

In [ ]:
# 5. 运行 LoRA 合并脚本
print("Starting LoRA merge...")
# 使用变量调用合并脚本
!python merge_lora.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}" --output_path "{merged_model_path}"

## 6. (可选) 转换为 GGUF 格式

现在我们将使用**合并后**的模型目录进行 GGUF 转换。
转换后的 GGUF 文件将保存在 `{gguf_output_abs_path}`。

In [ ]:
# 运行 LoRA 合并脚本
print("Starting LoRA merge...")
!python merge_lora.py --base_model_name "Qwen/Qwen2.5-1.5B-Instruct" --adapter_path "/content/echoheart_demo/output/qwen2-1.5b-qlora-ft" --output_path "/content/echoheart_demo/output/qwen2-1.5b-merged-ft"

# 转换为 GGUF
print("Starting GGUF conversion...")
!python convert_to_gguf.py --model_dir /content/echoheart_demo/output/qwen2-1.5b-merged-ft --output_file /content/echoheart_demo/output/qwen2-1.5b-merged-ft/gguf-model-f16.gguf --out_type f16